In [1]:
!pip install tabula-py

     |████████████████████████████████| 12.0 MB 88 kB/s 


In [2]:
import pandas as pd
from tabula import read_pdf
# [top,left,bottom,width]
box = [4,3,19,27]
fc = 28.28
for i in range(0, len(box)):
   box[i] *= fc

# **Prestamos de Misiones por Actividad**

In [3]:
def bcra(codigo):
    box = [4,3,19,26]
    fc = 28.28
    for i in range(0, len(box)):
      box[i] *= fc

    path = "http://www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/BoletinEstadistico/" + 'boldat' + str(codigo) +".pdf"
    tables = read_pdf(path, pages='314,316,318', area=[box], stream=True)
    
    misiones = pd.DataFrame()
    
    for tabla in tables:
      misiones = pd.concat(tables)
    misiones = misiones.rename(columns={'Unnamed: 0':'Actividad','Unnamed: 5':f'Misiones{str(codigo)}'})
    misiones = misiones[['Actividad',f'Misiones{str(codigo)}']]
    misiones['ID'] = range(len(misiones))
    misiones = misiones.set_index('ID')

    if codigo != 202001:
      misiones = misiones.drop(columns='Actividad')

    return misiones

In [4]:
def misiones(codes:list):
    primero = pd.concat([pd.DataFrame(bcra(202001))])
    definitivo = pd.concat([pd.DataFrame(bcra(i)) for i in codes], axis=1)
    definitivo2 = primero.merge(definitivo, how='left', on='ID') 
    return definitivo2

In [128]:
codes = [202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202106, 202107]
df = misiones(codes)
df.head()

,Actividad,Misiones202001,Misiones202002,Misiones202003,Misiones202004,Misiones202005,Misiones202006,Misiones202007,Misiones202008,Misiones202009,Misiones202010,Misiones202011,Misiones202012,Misiones202101,Misiones202102,Misiones202103,Misiones202104,Misiones202106,Misiones202107
ID,,,,,,,,,,,,,,,,,,,
0,Fabricación de papel y de productos de papel,"1,605","3,253","3,189","3,185","3,180","138,027","137,641","157,822","151,479","151,802","165,969","167,660","167,660","169,896","169,848","169,773","186,416","229,872"
1,Edición e impresión (32),"8,047","9,730","9,397","9,386","9,370","17,156,291","17,108,305","16,936,516","16,744,337","16,780,085","18,491,506","18,333,881","18,333,881","21,082,937","21,076,981","21,067,658","22,781,270","22,787,116"
2,Refinación del petróleo y comb. nuclear (33),"2,552","2,251","2,206","2,204","2,200","380,290","379,226","575,051","804,141","805,297","1,034,159","1,046,258","1,046,258","1,078,831","1,078,839","1,078,567","1,319,983","1,317,913"
3,Fabricación de prod. de caucho y plástico,"25,709","25,438","25,103","25,074","25,032","372,361","371,319","556,278","785,589","786,705","1,012,042","1,024,345","1,024,345","1,060,533","1,060,546","1,060,282","1,278,304","1,278,613"
4,Fabricación de prod. minerales no metálicos,"21,146","20,876","20,457","20,433","20,399","284,062","283,267","375,893","634,677","635,471","779,498","780,513","780,513","808,350","808,436","808,282","994,390","994,580"


# **Prestamos por Actividad todas las provincias**

## Funciones

In [35]:
box = [4,2.8,19,27]
fc = 28.28
for i in range(0, len(box)):
  box[i] *= fc

box2 = [4.8,2.8,19,27]
fc = 28.28
for i in range(0, len(box)):
  box2[i] *= fc

In [36]:
def provincias(codigo,pages:list,box):
    path = "http://www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/BoletinEstadistico/" + 'boldat' + str(codigo) +".pdf"
    tables = read_pdf(path, pages=pages, area=[box], stream=True)

    tablas1 = pd.DataFrame()
    tablas2 = pd.DataFrame()

    for page in pages:
      if page % 2 != 0:
          tablas1 = pd.concat(tables)
          tablas1 = tablas1.drop(columns=['(6)(7)','Unnamed: 1','Unnamed: 2','Unnamed: 4'])
          tablas1 = tablas1.rename(columns={'Unnamed: 0':'Actividad','Unnamed: 3':'Capital Federal','Aires':'Gran Buenos Aires','Unnamed: 5':'Resto Bs As',
                                          'Unnamed: 6':'Catamarca','Unnamed: 7':'Cordoba','Unnamed: 8':'Corrientes','Unnamed: 9':'Chaco','Unnamed: 10':'Chubut',
                                          'Unnamed: 11':'Entre Rios','Unnamed: 12':'Formosa','Unnamed: 13':'Jujuy'})
          tablas1['ID'] = range(len(tablas1))
          return tablas1

      else:
        for tabla in tables:
          if len(tabla.columns) <= 17:
            tabla = tabla.drop(columns=['(6)(7)','Unnamed: 1'])
            tabla = tabla.rename(columns={'Unnamed: 0':'Actividad','Unnamed: 2':'La Pampa','Unnamed: 3':'La Rioja','Unnamed: 4':'Mendoza',
                                            'Unnamed: 5':'Misiones','Unnamed: 6':'Neuquen','Unnamed: 7':'Rio Negro','Unnamed: 8':'Salta',
                                            'Unnamed: 9':'San Juan','Unnamed: 10':'San Luis','Unnamed: 11':'Santa Cruz','Unnamed: 12':'Santa Fe',
                                            'Estero':'Santiago del Estero','Fuego':'Tierra del Fuego','Unnamed: 13':'Tucuman'})
            tabla = pd.DataFrame(tabla)
            tablas2 = tablas2.append(tabla)

          elif len(tabla.columns) > 17:
            tabla = tabla.drop(columns=['(6)(7)','Unnamed: 1','Unnamed: 13'])
            tabla = tabla.rename(columns={'Unnamed: 0':'Actividad','Unnamed: 2':'La Pampa','Unnamed: 3':'La Rioja','Unnamed: 4':'Mendoza',
                                          'Unnamed: 5':'Misiones','Unnamed: 6':'Neuquen','Unnamed: 7':'Rio Negro','Unnamed: 8':'Salta',
                                          'Unnamed: 9':'San Juan','Unnamed: 10':'San Luis','Unnamed: 11':'Santa Cruz','Unnamed: 12':'Santa Fe',
                                          'Estero':'Santiago del Estero','Fuego':'Tierra del Fuego','Unnamed: 14':'Tucuman'})
            tabla = pd.DataFrame(tabla)
            tablas2 = tablas2.append(tabla)
          
            tablas2 = tablas2.drop(columns=['Actividad'])
            tablas2['ID'] = range(len(tablas2))
            return tablas2

In [37]:
def provincias2(codigo,pages:list,box):
  path = "http://www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/BoletinEstadistico/" + 'boldat' + str(codigo) +".pdf"
  tables = read_pdf(path, pages=pages, area=[box], stream=True)

  df = pd.concat(tables).reset_index(drop=True)
  
  if pages[0] % 2 != 0:
    df = df.drop(columns=['(6)(7)','Unnamed: 1','Unnamed: 2','Unnamed: 4'])
    df = df.rename(columns={'Unnamed: 0':'Actividad','Unnamed: 3':'Capital Federal','Aires':'Gran Buenos Aires','Unnamed: 5':'Resto Bs As',
                            'Unnamed: 6':'Catamarca','Unnamed: 7':'Cordoba','Unnamed: 8':'Corrientes','Unnamed: 9':'Chaco','Unnamed: 10':'Chubut',
                            'Unnamed: 11':'Entre Rios','Unnamed: 12':'Formosa','Unnamed: 13':'Jujuy'})
    return df
    
  else:
    df = df.drop(columns=['(6)(7)','Unnamed: 0','Unnamed: 1'])
    df = df.rename(columns={'Unnamed: 2':'La Pampa','Unnamed: 3':'La Rioja','Unnamed: 4':'Mendoza',
                            'Unnamed: 5':'Misiones','Unnamed: 6':'Neuquen','Unnamed: 7':'Rio Negro','Unnamed: 8':'Salta',
                            'Unnamed: 9':'San Juan','Unnamed: 10':'San Luis','Unnamed: 11':'Santa Cruz','Unnamed: 12':'Santa Fe',
                            'Estero':'Santiago del Estero','Fuego':'Tierra del Fuego','Unnamed: 13':'Tucuman'})
    
    return df

In [38]:
def whitespace_remover(df):
  for i in df.columns:
    if i != 'Actividad' and df[i].dtype == 'object':
      df[i] = df[i].str.replace(" ", "")
    else:
      pass

  return df

## Datos leídos

### Año 2019

In [70]:
enero2019 = provincias2(201901, pages=[307,309], box=box).merge(provincias2(201901, pages=[308,310], box=box), left_index=True, right_index=True)
febrero2019 = provincias2(201902, pages=[307,309], box=box).merge(provincias2(201902, pages=[308,310], box=box), left_index=True, right_index=True)
marzo2019 = provincias2(201903, pages=[307,309], box=box).merge(provincias2(201903, pages=[308,310], box=box), left_index=True, right_index=True)
abril2019 = provincias2(201904, pages=[307,309], box=box).merge(provincias2(201904, pages=[308,310], box=box), left_index=True, right_index=True)
mayo2019 = provincias2(201905, pages=[307,309], box=box).merge(provincias2(201905, pages=[308,310], box=box), left_index=True, right_index=True)
junio2019 = provincias2(201906, pages=[307,309], box=box).merge(provincias2(201906, pages=[308,310], box=box), left_index=True, right_index=True)
julio2019 = provincias2(201907, pages=[307,309], box=box).merge(provincias2(201907, pages=[308,310], box=box), left_index=True, right_index=True)
agosto2019 = provincias2(201908, pages=[307,309], box=box).merge(provincias2(201908, pages=[308,310], box=box), left_index=True, right_index=True)
septiembre2019 = provincias2(201909, pages=[307,309], box=box).merge(provincias2(201909, pages=[308,310], box=box), left_index=True, right_index=True)
octubre2019 = provincias2(201910, pages=[307,309], box=box).merge(provincias2(201910, pages=[308,310], box=box), left_index=True, right_index=True)
noviembre2019 = whitespace_remover(provincias2(201911, pages=[307,309], box=box2).merge(provincias2(201911, pages=[308,310], box=box2), left_index=True, right_index=True))
diciembre2019 = provincias(201912, pages=[307,309], box=box).merge(provincias(201912, pages=[308,310], box=box), how='left', on='ID').drop(columns=['ID'])

### Agregamos columnas años 2019

In [71]:
# Ejecutar una sola vez
enero2019['Año'] = 'ENE-2019'
cols = enero2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
enero2019 = enero2019[cols]

febrero2019['Año'] = 'FEB-2019'
cols = febrero2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
febrero2019 = febrero2019[cols]

marzo2019['Año'] = 'MAR-2019'
cols = marzo2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
marzo2019 = marzo2019[cols]

abril2019['Año'] = 2019
cols = abril2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
abril2019 = abril2019[cols]

mayo2019['Año'] = 'ABR-2019'
cols = mayo2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
mayo2019 = mayo2019[cols]

junio2019['Año'] = 'JUN-2019'
cols = junio2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
junio2019 = junio2019[cols]

julio2019['Año'] = 'JUL-2019'
cols = julio2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
julio2019 = julio2019[cols]

agosto2019['Año'] = 'AGO-2019'
cols = agosto2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
agosto2019 = agosto2019[cols]

septiembre2019['Año'] = 'SEP-2019'
cols = septiembre2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
septiembre2019 = septiembre2019[cols]

octubre2019['Año'] = 'OCT-2019'
cols = octubre2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
octubre2019 = octubre2019[cols]

noviembre2019['Año'] = 'NOV-2019'
cols = noviembre2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
noviembre2019 = noviembre2019[cols]

diciembre2019['Año'] = 'DIC-2019'
cols = diciembre2019.columns.tolist()
cols = cols[-1:] + cols[:-1]
diciembre2019 = diciembre2019[cols]

### Año 2020

In [72]:
enero2020 = provincias(202001, pages=[307,309], box=box).merge(provincias(202001, pages=[308,310], box=box), how='left', on='ID').drop(columns=['ID'])
febrero2020 = provincias(202002, pages=[307,309], box=box).merge(provincias(202002, pages=[308,310], box=box), how='left', on='ID').drop(columns=['ID'])
marzo2020 = provincias(202003, pages=[307,309], box=box).merge(provincias(202003, pages=[308,310], box=box), how='left', on='ID').drop(columns=['ID'])
abril2020 = provincias(202004, pages=[307,309], box=box).merge(provincias(202004, pages=[308,310], box=box), how='left', on='ID').drop(columns=['ID'])
mayo2020 = provincias(202005, pages=[307,309], box=box).merge(provincias(202005, pages=[308,310], box=box), how='left', on='ID').drop(columns=['ID'])
junio2020 = provincias(202006, pages=[309,311], box=box).merge(provincias(202006, pages=[310,312], box=box), how='left', on='ID').drop(columns=['ID'])
julio2020 = provincias(202007, pages=[309,311], box=box).merge(provincias(202007, pages=[310,312], box=box), how='left', on='ID').drop(columns=['ID'])
agosto2020 = provincias(202008, pages=[309,311], box=box).merge(provincias(202008, pages=[310,312], box=box), how='left', on='ID').drop(columns=['ID'])
septiembre2020 = provincias(202009, pages=[313,315], box=box).merge(provincias(202009, pages=[314,316], box=box), how='left', on='ID').drop(columns=['ID'])
octubre2020 = provincias(202010, pages=[313,315], box=box).merge(provincias(202010, pages=[314,316], box=box), how='left', on='ID').drop(columns=['ID'])
noviembre2020 = provincias(202011, pages=[313,315], box=box).merge(provincias(202011, pages=[314,316], box=box), how='left', on='ID').drop(columns=['ID'])
diciembre2020 = provincias(202012, pages=[313,315], box=box).merge(provincias(202012, pages=[314,316], box=box), how='left', on='ID').drop(columns=['ID'])

### Agregamos columnas años 2020

In [73]:
enero2020['Año'] = 'ENE-2020'
cols = enero2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
enero2020 = enero2020[cols]

febrero2020['Año'] = 'FEB-2020'
cols = febrero2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
febrero2020 = febrero2020[cols]

marzo2020['Año'] = 'MAR-2020'
cols = marzo2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
marzo2020 = marzo2020[cols]

abril2020['Año'] = 'ABR-2020'
cols = abril2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
abril2020 = abril2020[cols]

mayo2020['Año'] = 'MAY-2020'
cols = mayo2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
mayo2020 = mayo2020[cols]

junio2020['Año'] = 'JUN-2020'
cols = junio2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
junio2020 = junio2020[cols]

julio2020['Año'] = 'JUL-2020'
cols = julio2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
julio2020 = julio2020[cols]

agosto2020['Año'] = 'AGO-2020'
cols = agosto2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
agosto2020 = agosto2020[cols]

septiembre2020['Año'] = 'SEP-2020'
cols = septiembre2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
septiembre2020 = septiembre2020[cols]

octubre2020['Año'] = 'OCT-2020'
cols = octubre2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
octubre2020 = octubre2020[cols]

noviembre2020['Año'] = 'NOV-2020'
cols = noviembre2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
noviembre2020 = noviembre2020[cols]

diciembre2020['Año'] = 'DIC-2020'
cols = diciembre2020.columns.tolist()
cols = cols[-1:] + cols[:-1]
diciembre2020 = diciembre2020[cols]

### Año 2021

In [74]:
enero2021 = provincias(202101, pages=[313,315], box=box).merge(provincias(202101, pages=[314,316], box=box), how='left', on='ID').drop(columns=['ID'])
febrero2021 = provincias(202102, pages=[313,315], box=box).merge(provincias(202102, pages=[314,316], box=box), how='left', on='ID').drop(columns=['ID'])
marzo2021 = provincias(202103, pages=[313,315], box=box).merge(provincias(202103, pages=[314,316], box=box), how='left', on='ID').drop(columns=['ID'])
abril2021 = provincias(202104, pages=[313,315], box=box).merge(provincias(202104, pages=[314,316], box=box), how='left', on='ID').drop(columns=['ID'])
mayo2021 = provincias2(202105, pages=[313,315], box=box2).merge(provincias2(202105, pages=[314,316], box=box2), left_index=True, right_index=True)
junio2021 = provincias2(202106, pages=[313,315], box=box).merge(provincias2(202106, pages=[314,316], box=box), left_index=True, right_index=True)

### Agregamos columnas años 2021

In [75]:
enero2021['Año'] = 'ENE-2021'
cols = enero2021.columns.tolist()
cols = cols[-1:] + cols[:-1]
enero2021 = enero2021[cols]

febrero2021['Año'] = 'FEB-2021'
cols = febrero2021.columns.tolist()
cols = cols[-1:] + cols[:-1]
febrero2021 = febrero2021[cols]

marzo2021['Año'] = 'MAR-2021'
cols = marzo2021.columns.tolist()
cols = cols[-1:] + cols[:-1]
marzo2020 = marzo2021[cols]

abril2021['Año'] = 'ABR-2021'
cols = abril2021.columns.tolist()
cols = cols[-1:] + cols[:-1]
abril2021 = abril2021[cols]

mayo2021['Año'] = 'MAY-2021'
cols = mayo2021.columns.tolist()
cols = cols[-1:] + cols[:-1]
mayo2021 = mayo2021[cols]

junio2021['Año'] = 'JUN-2021'
cols = junio2021.columns.tolist()
cols = cols[-1:] + cols[:-1]
junio2021 = junio2021[cols]

## DataFrame con todos los datos

In [76]:
dfs = [enero2019,febrero2019,marzo2019,abril2019,mayo2019,junio2019,julio2019,agosto2019,septiembre2019,octubre2019,noviembre2019,diciembre2019,
       enero2020,febrero2020,marzo2020,abril2020,mayo2020,junio2020,julio2020,agosto2020,septiembre2020,octubre2020,noviembre2020,diciembre2020,
       enero2021,febrero2021,marzo2021,abril2021,mayo2021,junio2021]

In [77]:
dfs = pd.concat(dfs)
dfs

,Año,Actividad,Capital Federal,Gran Buenos Aires,Resto Bs As,Catamarca,Cordoba,Corrientes,Chaco,Chubut,Entre Rios,Formosa,Jujuy,La Pampa,La Rioja,Mendoza,Misiones,Neuquen,Rio Negro,Salta,San Juan,San Luis,Santa Cruz,Santa Fe,Santiago del Estero,Tierra del Fuego,Tucuman
0,ENE-2019,No identificada,"30,069,924","11,796,965","14,046,162","83,791","6,102,017","159,271","174,776","308,331","128,654","9,600","119,602","107,181","17,276","769,439","115,288","2,385,963","223,116","276,196","220,340","156,459","78,805","1,303,630","118,698","252,768","408,254"
1,ENE-2019,Personas físicas en relación de dependencia la...,"222,066,393","111,024,612","104,464,212","3,596,091","49,631,625","8,520,360","9,743,733","14,318,083","12,549,253","5,976,194","8,099,063","5,997,674","3,197,867","21,686,074","15,416,962","15,933,633","8,709,404","19,940,227","4,661,007","3,885,331","6,496,933","49,882,060","10,275,290","7,703,433","21,806,137"
2,ENE-2019,Producción primaria,"176,365,002","9,491,460","42,537,520","247,334","33,258,390","2,583,919","2,230,516","2,192,733","6,038,677","145,373","792,622","3,878,346","610,502","6,825,430","914,558","5,164,677","2,417,923","4,435,646","897,651","1,697,969","282,585","28,099,698","507,765","36,468","6,706,569"
3,ENE-2019,"Agricultura, ganadería, caza y silvicultura","81,920,775","8,823,600","40,997,748","228,950","32,588,884","2,538,254","2,217,512","463,541","5,948,175","145,237","576,212","3,822,070","605,371","6,037,213","895,188","285,662","2,296,239","4,357,665","568,802","1,617,634","83,362","27,585,338","479,968","9,215","6,259,603"
4,ENE-2019,Cultivos y servicios agrícolas,"48,521,125","5,017,330","30,189,952","217,921","26,280,829","399,428","1,793,665","291,400","3,794,136","33,470","541,680","2,018,726","602,302","3,742,709","665,481","186,641","2,081,179","3,621,807","465,747","360,915","27,718","22,805,216","435,623","3,955","4,740,486"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,JUN-2021,Esparcimiento y servicios culturales y deportivos,"4,350,703","452,164","378,669","1,665","247,432","46,881","19,729","33,642","163,186","702,974","21,312","6,692","2,147","133,538","67,035","115,455","45,785","48,547","967,668","5,021","577,392","292,841","8,664","36,517","31,016"
114,JUN-2021,Otras asociaciones,"3,069,426","357,423","153,583","1,496","239,730","32,348","1,733,762","44,860","26,677","7,132","6,685","25,913","4,855","242,065","10,983","290,864","23,825","46,585","18,502","1,947","19,438","273,344","2,677","12,120","335,868"
115,JUN-2021,Organizaciones y órganos extraterritoriales,"11,955","2,347","3,152",-,"1,181",-,-,82,"1,366","1,116",567,-,-,64,-,667,39,612,23,136,90,"39,760",1,2,91
116,JUN-2021,Otros servicios,"58,302,426","11,741,750","11,040,025","229,887","9,408,918","869,007","1,117,516","1,242,438","2,893,696","247,511","232,584","1,368,893","760,596","3,274,338","828,513","1,567,795","983,144","697,297","809,220","630,754","675,328","10,783,812","360,024","486,311","3,075,472"


In [84]:
dfs.to_csv('/content/drive/MyDrive/Ministerio del Agro/Prestamos BCRA/PrestamosMES', sep=';')

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
for i in dfs.columns:
    if i == 'Año':
      dfs[i] = dfs[i].str.replace("ENE-2019","01-2019")
      dfs[i] = dfs[i].str.replace("FEB-2019","02-2019")
      dfs[i] = dfs[i].str.replace("MAR-2019","03-2019")
      dfs[i] = dfs[i].str.replace("ABR-2019","04-2019")
      dfs[i] = dfs[i].str.replace("MAY-2019","05-2019")
      dfs[i] = dfs[i].str.replace("JUN-2019","06-2019")
      dfs[i] = dfs[i].str.replace("JUL-2019","07-2019")
      dfs[i] = dfs[i].str.replace("AGO-2019","08-2019")
      dfs[i] = dfs[i].str.replace("SEP-2019","09-2019")
      dfs[i] = dfs[i].str.replace("OCT-2019","10-2019")
      dfs[i] = dfs[i].str.replace("NOV-2019","11-2019")
      dfs[i] = dfs[i].str.replace("DIC-2019","12-2019")
    else:
      pass

In [81]:
for i in dfs.columns:
    if i == 'Año':
      dfs[i] = dfs[i].str.replace("ENE-2020","01-2020")
      dfs[i] = dfs[i].str.replace("FEB-2020","02-2020")
      dfs[i] = dfs[i].str.replace("MAR-2020","03-2020")
      dfs[i] = dfs[i].str.replace("ABR-2020","04-2020")
      dfs[i] = dfs[i].str.replace("MAY-2020","05-2020")
      dfs[i] = dfs[i].str.replace("JUN-2020","06-2020")
      dfs[i] = dfs[i].str.replace("JUL-2020","07-2020")
      dfs[i] = dfs[i].str.replace("AGO-2020","08-2020")
      dfs[i] = dfs[i].str.replace("SEP-2020","09-2020")
      dfs[i] = dfs[i].str.replace("OCT-2020","10-2020")
      dfs[i] = dfs[i].str.replace("NOV-2020","11-2020")
      dfs[i] = dfs[i].str.replace("DIC-2020","12-2020")
    else:
      pass

In [82]:
for i in dfs.columns:
    if i == 'Año':
      dfs[i] = dfs[i].str.replace("ENE-2021","01-2021")
      dfs[i] = dfs[i].str.replace("FEB-2021","02-2021")
      dfs[i] = dfs[i].str.replace("MAR-2021","03-2021")
      dfs[i] = dfs[i].str.replace("ABR-2021","04-2021")
      dfs[i] = dfs[i].str.replace("MAY-2021","05-2021")
      dfs[i] = dfs[i].str.replace("JUN-2021","06-2021")
    else:
      pass

In [83]:
dfs

,Año,Actividad,Capital Federal,Gran Buenos Aires,Resto Bs As,Catamarca,Cordoba,Corrientes,Chaco,Chubut,Entre Rios,Formosa,Jujuy,La Pampa,La Rioja,Mendoza,Misiones,Neuquen,Rio Negro,Salta,San Juan,San Luis,Santa Cruz,Santa Fe,Santiago del Estero,Tierra del Fuego,Tucuman
0,01-2019,No identificada,"30,069,924","11,796,965","14,046,162","83,791","6,102,017","159,271","174,776","308,331","128,654","9,600","119,602","107,181","17,276","769,439","115,288","2,385,963","223,116","276,196","220,340","156,459","78,805","1,303,630","118,698","252,768","408,254"
1,01-2019,Personas físicas en relación de dependencia la...,"222,066,393","111,024,612","104,464,212","3,596,091","49,631,625","8,520,360","9,743,733","14,318,083","12,549,253","5,976,194","8,099,063","5,997,674","3,197,867","21,686,074","15,416,962","15,933,633","8,709,404","19,940,227","4,661,007","3,885,331","6,496,933","49,882,060","10,275,290","7,703,433","21,806,137"
2,01-2019,Producción primaria,"176,365,002","9,491,460","42,537,520","247,334","33,258,390","2,583,919","2,230,516","2,192,733","6,038,677","145,373","792,622","3,878,346","610,502","6,825,430","914,558","5,164,677","2,417,923","4,435,646","897,651","1,697,969","282,585","28,099,698","507,765","36,468","6,706,569"
3,01-2019,"Agricultura, ganadería, caza y silvicultura","81,920,775","8,823,600","40,997,748","228,950","32,588,884","2,538,254","2,217,512","463,541","5,948,175","145,237","576,212","3,822,070","605,371","6,037,213","895,188","285,662","2,296,239","4,357,665","568,802","1,617,634","83,362","27,585,338","479,968","9,215","6,259,603"
4,01-2019,Cultivos y servicios agrícolas,"48,521,125","5,017,330","30,189,952","217,921","26,280,829","399,428","1,793,665","291,400","3,794,136","33,470","541,680","2,018,726","602,302","3,742,709","665,481","186,641","2,081,179","3,621,807","465,747","360,915","27,718","22,805,216","435,623","3,955","4,740,486"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,06-2021,Esparcimiento y servicios culturales y deportivos,"4,350,703","452,164","378,669","1,665","247,432","46,881","19,729","33,642","163,186","702,974","21,312","6,692","2,147","133,538","67,035","115,455","45,785","48,547","967,668","5,021","577,392","292,841","8,664","36,517","31,016"
114,06-2021,Otras asociaciones,"3,069,426","357,423","153,583","1,496","239,730","32,348","1,733,762","44,860","26,677","7,132","6,685","25,913","4,855","242,065","10,983","290,864","23,825","46,585","18,502","1,947","19,438","273,344","2,677","12,120","335,868"
115,06-2021,Organizaciones y órganos extraterritoriales,"11,955","2,347","3,152",-,"1,181",-,-,82,"1,366","1,116",567,-,-,64,-,667,39,612,23,136,90,"39,760",1,2,91
116,06-2021,Otros servicios,"58,302,426","11,741,750","11,040,025","229,887","9,408,918","869,007","1,117,516","1,242,438","2,893,696","247,511","232,584","1,368,893","760,596","3,274,338","828,513","1,567,795","983,144","697,297","809,220","630,754","675,328","10,783,812","360,024","486,311","3,075,472"


# **Alphacast**

In [55]:
!pip install alphacast

In [56]:
from alphacast import Alphacast
alphacast = Alphacast('ak_JfHRRcvvlfnEGAK8YXta')

In [159]:
alphacast.repository.read_all()

[{'accountId': 708,
  'description': 'You can use you first public repository to share content with the community',
  'id': 1002,
  'name': 'My Public Repo',
  'permission': 'Owner',
  'privacy': 'Public',
  'slug': 'public-repo'},
 {'accountId': 708,
  'description': 'Escrapeos y transformación de datos realizados por el Licenciado en Economía Juan Pablo Benitez para la vacante laboral Data Analyst',
  'id': 1243,
  'name': 'Vacante Data Analyst',
  'permission': 'Owner',
  'privacy': 'Private',
  'slug': 'vacante-data-analyst'},
 {'accountId': 708,
  'description': 'Repositorio para el puesto DAta Analyst',
  'id': 1245,
  'name': 'Repo Juan Benitez',
  'permission': 'Owner',
  'privacy': 'Public',
  'slug': 'repo-juan-benitez'},
 {'accountId': 708,
  'description': 'Repositorio para el puesto Data Analyst',
  'id': 1246,
  'name': 'Repo Juan Benitez1',
  'permission': 'Owner',
  'privacy': 'Public',
  'slug': 'repo-juan-benitez1'},
 {'accountId': 708,
  'description': 'Repositorio p

In [160]:
alphacast.datasets.create('Préstamo BCRA Produccion Primaria', 1249, 'Scrapping de datos de préstamos del BCRA')

{'createdAt': '2021-10-16T03:42:55.055281',
 'id': 8106,
 'name': 'Préstamo BCRA Produccion Primaria',
 'repositoryId': 1249}

In [161]:
alphacast.datasets.create('Préstamo BCRA Misiones por Actividad', 1249, 'Scrapping de datos de préstamos del BCRA')

{'createdAt': '2021-10-16T03:43:03.687917',
 'id': 8107,
 'name': 'Préstamo BCRA Misiones por Actividad',
 'repositoryId': 1249}

In [162]:
df2 = df
df2 = df2.drop(columns={'level_0','index','ID'})
# df2 = df2.transpose()
columnas = df2['Actividad'].to_list()
df2 = df2.transpose()
df2.columns = columnas
df2 = df2.reset_index()
df2 = df2.drop(0)
df2 = df2.rename(columns={'index':'Date'})

for i in df2.columns:
    if i == 'Date':
      df2[i] = df2[i].str.replace("Misiones202001","01-2020")
      df2[i] = df2[i].str.replace("Misiones202002","02-2020")
      df2[i] = df2[i].str.replace("Misiones202003","03-2020")
      df2[i] = df2[i].str.replace("Misiones202004","04-2020")
      df2[i] = df2[i].str.replace("Misiones202005","05-2020")
      df2[i] = df2[i].str.replace("Misiones202006","06-2020")
      df2[i] = df2[i].str.replace("Misiones202007","07-2020")
      df2[i] = df2[i].str.replace("Misiones202008","08-2020")
      df2[i] = df2[i].str.replace("Misiones202009","09-2020")
      df2[i] = df2[i].str.replace("Misiones202010","10-2020")
      df2[i] = df2[i].str.replace("Misiones202011","11-2020")
      df2[i] = df2[i].str.replace("Misiones202012","12-2020")
      df2[i] = df2[i].str.replace("Misiones202101","01-2021")
      df2[i] = df2[i].str.replace("Misiones202102","02-2021")
      df2[i] = df2[i].str.replace("Misiones202103","03-2021")
      df2[i] = df2[i].str.replace("Misiones202104","04-2021")
      df2[i] = df2[i].str.replace("Misiones202105","05-2021")
      df2[i] = df2[i].str.replace("Misiones202106","06-2021")
      df2[i] = df2[i].str.replace("Misiones202107","07-2021")
    else:
      pass
    
df2['Periodo'] = range(0,len(df2))
df2

,Date,Fabricación de papel y de productos de papel,Edición e impresión (32),Refinación del petróleo y comb. nuclear (33),Fabricación de prod. de caucho y plástico,Fabricación de prod. minerales no metálicos,Fabricación de metales comunes,Fabricación de prod. elaborados de metal (34),Reciclamiento,Otras industrias manufactureras (35),"Electricidad, gas y agua","Electricidad, gas, vapor y agua caliente",Energía eléctrica (36),Gas (37),Suministro de vapor y agua caliente,"Captación, depuración y distribución de agua",Construcción,Comercio al por mayor y al por menor (38),"Venta, mantenimiento y reparación de vehículos (39)",Comercio al por mayor (40),Comercio al por menor (41),Servicios,Hotelería y restaurantes,"Transportes, almacenamiento y comunicaciones",Transporte terrestre,Transporte por vía acuática,Transporte aéreo,Servicios anexos al transporte (42),Correos y telecomunicaciones,Intermediación financiera y otros servicios financ.,Banco Central de la República Argentina,Entidades financieras autorizadas por el BCRA,Seguros y admin. de fondos de jubil. y pens.,Servicios auxiliares a la actividad financiera,Otros servicios financieros,"Inmobiliarios, empresariales y de alquiler",Inmobiliarios,Alquiler de equipo de transporte,Informáticos y actividades conexas,Investigación y desarrollo,...,Gas (37),Suministro de vapor y agua caliente,"Captación, depuración y distribución de agua",Construcción,Comercio al por mayor y al por menor (38),"Venta, mantenimiento y reparación de vehículos (39)",Comercio al por mayor (40),Comercio al por menor (41),Servicios,Hotelería y restaurantes,"Transportes, almacenamiento y comunicaciones",Transporte terrestre,Transporte por vía acuática,Transporte aéreo,Servicios anexos al transporte (42),Correos y telecomunicaciones,Intermediación financiera y otros servicios financ.,Banco Central de la República Argentina,Entidades financieras autorizadas por el BCRA,Seguros y admin. de fondos de jubil. y pens.,Servicios auxiliares a la actividad financiera,Otros servicios financieros,"Inmobiliarios, empresariales y de alquiler",Inmobiliarios,Alquiler de equipo de transporte,Informáticos y actividades conexas,Investigación y desarrollo,Otros servicios empresariales,"Administ. pública, defensa y seguridad social oblig.","Enseñanza, servicios sociales y de salud",Enseñanza,Servicios sociales y de salud,Otros,Elimin. de desperdicios y aguas residuales (43),Esparcimiento y servicios culturales y deportivos,Otras asociaciones,Organizaciones y órganos extraterritoriales,Otros servicios,Total de actividades económicas,Periodo
1,01-2020,"1,605","8,047","2,552","25,709","21,146","8,009","29,168",-,"10,640","46,318","46,164","43,003","3,122",38,154,"491,492","1,035,589","144,104","351,698","539,787","1,909,827","23,582","318,768","276,016","13,928",573,"22,311","5,938","24,506",-,"2,091","8,474","2,877","11,064","267,972","16,496","1,318","8,484",739,...,-,-,-,"3,031","90,297",286,"80,461","9,550","15,040",629,"2,002",629,-,-,"1,372",-,"1,772",-,-,629,-,"1,144","3,488",687,-,58,-,"2,745",400,"1,602",286,"1,315","5,146",-,58,-,-,"5,090","2,419,880",0
2,02-2020,"3,253","9,730","2,251","25,438","20,876","7,277","28,218",-,"11,395","46,371","46,212","43,086","3,116",10,159,"505,956","1,074,517","147,418","377,495","549,604","1,962,488","27,793","309,175","262,317","12,719",588,"28,330","5,221","26,512",-,"2,062","9,965","3,111","11,375","276,736","19,149","1,070","10,147","1,092",...,-,-,-,391,"87,958",224,"78,613","9,120","17,513",447,"2,294",839,-,-,"1,455",-,"1,398",-,-,279,-,"1,119","4,196",391,-,-,-,"3,805","1,398","2,182",504,"1,679","5,596",-,112,-,-,"5,484","1,565,502",1
3,03-2020,"3,189","9,397","2,206","25,103","20,457","7,133","27,572",-,"11,072","49,078","48,932","46,196","2,725",10,146,"496,632","1,048,347","133,687","367,865","546,795","1,916,831","26,618","302,544","255,549","12,527",960,"28,268","5,240","26,535",-,"2,228","9,554","3,034","11,719","271,083","18,802","1,049","9,822","1,070",...,-,-,-,531,"93,

In [163]:
# df1 = dfs.drop(dfs['Actividad'=='Total de actividades económicas'])
# df1
df1 = dfs
df1 = df1.loc[df1['Actividad'] == 'Producción primaria']
df1 = df1.drop(columns='Actividad')
df1['Periodo'] = range(len(df1))
df1

,Año,Capital Federal,Gran Buenos Aires,Resto Bs As,Catamarca,Cordoba,Corrientes,Chaco,Chubut,Entre Rios,Formosa,Jujuy,La Pampa,La Rioja,Mendoza,Misiones,Neuquen,Rio Negro,Salta,San Juan,San Luis,Santa Cruz,Santa Fe,Santiago del Estero,Tierra del Fuego,Tucuman,Periodo
2,01-2019,"176,365,002","9,491,460","42,537,520","247,334","33,258,390","2,583,919","2,230,516","2,192,733","6,038,677","145,373","792,622","3,878,346","610,502","6,825,430","914,558","5,164,677","2,417,923","4,435,646","897,651","1,697,969","282,585","28,099,698","507,765","36,468","6,706,569",0
2,02-2019,"162,325,673","8,974,158","39,440,992","268,352","32,317,950","2,418,859","2,209,535","2,199,814","5,411,183","140,177","790,529","3,777,770","549,991","6,506,331","812,406","4,886,454","2,449,274","4,133,034","1,048,951","1,506,884","279,694","27,117,406","497,793","41,797","5,671,894",1
2,03-2019,"161,801,135","8,472,979","39,540,384","261,143","31,791,454","2,400,488","2,128,701","2,178,613","5,402,482","140,495","870,704","3,780,579","546,030","6,510,072","792,854","4,840,331","2,443,592","4,379,862","1,043,443","1,472,270","281,636","27,809,506","497,163","42,323","5,693,395",2
2,NaN,"160,995,885","8,565,162","39,422,420","261,349","31,618,771","2,432,366","2,180,896","2,275,232","5,407,062","139,656","866,566","3,782,685","543,675","6,407,597","788,951","4,820,749","2,441,004","4,356,618","1,040,370","1,466,733","267,187","40,556,307","510,731","41,984","5,663,664",3
2,04-2019,"173,880,157","10,163,795","40,679,307","291,827","33,609,123","2,471,382","2,224,320","2,628,491","6,353,684","144,181","898,632","3,613,985","638,866","6,781,455","800,011","4,494,579","2,759,861","5,113,578","1,286,700","2,556,178","264,341","44,034,473","526,054","45,578","5,717,589",4
2,06-2019,"175,953,463","10,176,008","40,603,231","288,654","33,714,071","2,444,291","2,180,726","2,601,851","6,303,384","147,085","878,994","3,658,264","636,532","6,715,932","766,394","4,462,914","2,808,834","4,937,107","1,319,444","2,597,864","266,150","43,806,490","505,964","44,700","5,519,968",5
2,07-2019,"175,994,650","10,163,352","40,618,463","288,749","33,729,121","2,445,376","2,181,813","2,602,791","6,311,913","147,195","879,297","3,660,775","636,676","6,718,488","766,659","4,463,632","2,809,366","4,937,748","1,319,970","2,599,069","266,355","43,811,839","506,229","44,734","5,521,314",6
2,08-2019,"171,180,103","11,351,016","38,889,458","273,408","31,501,557","2,490,234","2,789,994","2,532,581","5,183,980","135,731","1,067,735","3,472,832","562,577","6,537,938","719,127","7,347,554","2,715,768","4,692,842","1,262,197","2,698,954","266,627","42,340,815","498,043","124,342","4,817,335",7
2,09-2019,"178,198,536","13,690,523","38,809,136","273,931","31,526,420","2,492,088","2,790,264","2,555,863","5,209,753","135,823","1,061,475","3,478,037","564,331","7,451,214","719,753","7,338,532","2,687,497","4,761,020","1,277,897","2,715,164","259,743","42,316,878","503,634","124,687","4,910,242",8
2,10-2019,"178,197,110","13,690,251","38,808,106","273,925","31,525,722","2,492,014","2,790,187","2,555,796","5,209,573","135,817","1,061,458","3,477,884","564,323","7,451,789","719,737","7,338,480","2,687,462","4,760,971","1,279,574","2,715,083","259,730","42,316,283","503,617","124,684","4,910,148",9


In [164]:
alphacast.datasets.dataset(8106).initialize_columns(dateColumnName ="Año", entitiesColumnNames=["Periodo"], dateFormat= "%m-%Y")

b'{"id": 8106, "columnDefinitions": [{"sourceName": "A\\u00f1o", "dataType": "Date", "dateFormat": "%m-%Y", "isEntity": "True"}, {"sourceName": "Periodo", "isEntity": "True"}], "updateAt": "2021-10-16T03:44:09.894111"}'

In [165]:
alphacast.datasets.dataset(8106).upload_data_from_df(df1, deleteMissingFromDB = False, onConflictUpdateDB = False, uploadIndex=False)

b'{"id": 54842, "status": "Requested", "createdAt": "2021-10-16T03:44:12.011735", "datasetId": 8106}'

In [166]:
alphacast.datasets.dataset(8107).initialize_columns(dateColumnName ="Date", entitiesColumnNames=["Periodo"], dateFormat= "%m-%Y")

b'{"id": 8107, "columnDefinitions": [{"sourceName": "Date", "dataType": "Date", "dateFormat": "%m-%Y", "isEntity": "True"}, {"sourceName": "Periodo", "isEntity": "True"}], "updateAt": "2021-10-16T03:44:19.619171"}'

In [167]:
alphacast.datasets.dataset(8107).upload_data_from_df(df2, deleteMissingFromDB = False, onConflictUpdateDB = False, uploadIndex=False)

b'{"id": 54843, "status": "Requested", "createdAt": "2021-10-16T03:44:24.513649", "datasetId": 8107}'